In [1]:
import requests
import re 
import time
import os
from selenium import webdriver
from pyquery import PyQuery as pq
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [193]:
root_url = 'https://www1.hkexnews.hk/search/titlesearch.xhtml'

url_17 = '?lang=ZH&category=0&market=SEHK&searchType=1&documentType=&t1code=40000&t2Gcode=-2&t2code=-2&stockId=-1&from=20170101&to=20180101&MB-Daterange=0&title='
url_18 = '?lang=ZH&category=0&market=SEHK&searchType=1&documentType=&t1code=40000&t2Gcode=-2&t2code=-2&stockId=-1&from=20180101&to=20190101&MB-Daterange=0&title='
url_19 = '?lang=ZH&category=0&market=SEHK&searchType=1&documentType=&t1code=40000&t2Gcode=-2&t2code=-2&stockId=-1&from=20190101&to=20200101&MB-Daterange=0&title='
url_20 = '?lang=ZH&category=0&market=SEHK&searchType=1&documentType=&t1code=40000&t2Gcode=-2&t2code=-2&stockId=-1&from=20200101&to=20210101&MB-Daterange=0&title='

In [194]:
#准备工作，将四年搜索的url尾缀存在url_list中，方便后续拼接
url_list = []
url_num_list = [17,18,19,20]

for i in url_num_list:
    url_list.append(eval('url_'+str(i)))

In [195]:
for num in range(len(url_num_list)):
    url = root_url + url_list[num]   #url拼接
    browser  = webdriver.Chrome()  #打开chrome
    browser.get(url)  #输入url
    html=browser.page_source
    data=str(pq(html))  #data为页面信息
    
    total_num_str = browser.find_element_by_class_name("component-loadmore-leftPart__container").text
    #total_num_str = '顯示500紀錄，共有14239紀錄' 准备求总记录数量
    num_list = re.findall('\d+', total_num_str)
    #num_list = ['500', '14239']
    
    while num_list[0]!=num_list[1]:
        try:
            wait = WebDriverWait(browser, 10)
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'component-loadmore__link')))
            browser.find_element_by_class_name('component-loadmore__link').click()
            time.sleep(1)
        except:
            time.sleep(1)
        total_num_str = browser.find_element_by_class_name("component-loadmore-leftPart__container").text
        num_list = re.findall('\d+', total_num_str)
    #点击加载全部记录
    
    re_rule1 = '<div class="doc-link">.*?a href="(.*?)" rel=".*?" target=".*?">'

    datalist=re.findall(re_rule1,data,re.S)
    #定位pdf文件url，用datalist存储
    
    re_rule2 = '<div class="doc-link">.*?a href=".*?" rel=".*?" target=".*?">(.*?)</a>'

    namelist=re.findall(re_rule2,data,re.S)

    for i in range(len(namelist)):
        namelist[i] = namelist[i].replace('\n','').replace('\t','').replace('/','_').strip().replace(' ','_')
    #存储pdf文件名
    
    if not os.path.exists(str(url_num_list[num])) :
        os.mkdir(str(url_num_list[num]))
    os.chdir(os.path.join(os.getcwd(), str(url_num_list[num])))
    #将存储路径改为17,18,19,20文件夹

    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OSX10_14_2) AppleWebKit/537.36(KHTML, like Gecko) Chrome/ 96.0.4664.45 Safari/537.36'
    }

    if len(namelist)==len(datalist):
        for i in range(len(namelist)):
            data_url = 'https://www1.hkexnews.hk/'+datalist[i]
            pdf_name = namelist[i]+'.pdf'
            response = requests.get(data_url,headers=headers)
            with open(pdf_name, 'wb') as f: 
                f.write(response.content)
            f.close()
    else:
        print('lenth not equal')
    #保存pdf
    os.chdir(os.path.join(os.getcwd(), '..'))
    #回退路径，以免导致多层文件夹

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.55)


In [167]:
os.chdir(os.path.join(os.getcwd(), '..'))